In [ ]:
import requests as rq
import spotipy as spot
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import time
import matplotlib.pyplot as plt
from IPython.display import Markdown as md

In [ ]:
SPOTIFY_CLIENT_ID = 'e81e503997aa4f318c13abbe6a7d623b'
SPOTIFY_CLIENT_SECRET = '###' #censored for security
SPOTIFY_REDIRECT_URI = 'http://127.0.0.1:9090'
SCOPE ='user-top-read, user-read-private'

In [ ]:
sp = spot.Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIFY_CLIENT_ID,
                                            client_secret=SPOTIFY_CLIENT_SECRET,
                                            redirect_uri=SPOTIFY_REDIRECT_URI,
                                            scope=SCOPE))

In [ ]:
top_tracks = sp.current_user_top_tracks(limit=25, offset=0, time_range='long_term')

In [ ]:
def get_track_ids(time_frame, key_word):
    track_ids = []
    for song in time_frame[key_word]:
        track_ids.append(song['id'])
    return track_ids

In [ ]:
track_ids = get_track_ids(top_tracks, 'items')

In [ ]:
def get_track_data(track_id):
    meta = sp.track(track_id)
    return [meta['name'],
            meta['album']['name'],
            meta['album']['artists'][0]['name'],
            meta['album']['artists'][0]['id'],
            meta['external_urls']['spotify'],
            meta['album']['images'][0]['url']]

In [ ]:
def get_track_features(track_id):
    features = sp.audio_features(track_id)
    return [features[0]['danceability'],
            features[0]['energy'],
            features[0]['key'],
            features[0]['loudness'],
            features[0]['tempo'],
            features[0]['time_signature']]

In [ ]:
tracks = []
for i in range(len(track_ids)):
    time.sleep(.5)
    tracks.append(get_track_data(track_ids[i])+get_track_features(track_ids[i]))

In [ ]:
df = pd.DataFrame(tracks, columns=['name', 'album', 
                                   'artist', 'artist_id', 'spotify_url', 
                                   'album_cover', 'danceability', 
                                   'energy', 'key',
                                   'loudness', 'tempo',
                                   'time_signature'])
df.head(25)

In [ ]:
_, ax = plt.subplots(1, 1)
ax.hist(df['energy'], bins = 5, color = '#FF9300')
ax.set_title("Energy Distribution")

ax.set_xlabel('energy')
ax.set_ylabel('frequency')

plt.show()

In [ ]:
_, ax = plt.subplots(1, 1)
ax.hist(df['tempo'], bins = 5)
ax.set_title("Tempo Distribution")

ax.set_xlabel('tempo')
ax.set_ylabel('frequency')

plt.show()

In [ ]:
_, ax = plt.subplots(1, 1)
ax.hist(df['key'], bins = [i for i in range(-1, 12)], color = '#FFEC50')
ax.set_title("Key Distribution")

ax.set_xlabel('key')
ax.set_ylabel('frequency')

plt.show()

In [ ]:
top_album = df.loc[df['album'] == df['album'].mode()[0]]
top_album.reset_index(inplace=True)
top_album_info = {'name': top_album['album'][0],
                  'artist': top_album['artist'][0],
                  'cover': top_album['album_cover'][0]}

In [ ]:
top_img = rq.get(top_album_info['cover']).content
with open('top_album_cover.jpg', 'wb') as handler:
    handler.write(top_img)

In [ ]:
md(f"My top album is *{top_album_info['name']}* by *{top_album_info['artist']}*")

![top album cover](top_album_cover.jpg)

In [ ]:
top_artist = df.loc[df['artist'] == df['artist'].mode()[0]]
top_artist.reset_index(inplace=True)
top_artist_id = top_artist['artist_id'][0]

In [ ]:
def get_artist_info(artist_id):
    artist_info = sp.artist(artist_id)
    return {'name': artist_info['name'],
            'popularity': artist_info['popularity'],
            'followers': artist_info['followers']['total'],
            'image': artist_info['images'][0]['url']}

In [ ]:
top_artist_info = get_artist_info(top_artist_id)

In [ ]:
top_img = rq.get(top_artist_info['image']).content
with open('top_artist_photo.jpg', 'wb') as handler:
    handler.write(top_img)

In [ ]:
md(f"My top artist is *{top_artist_info['name']}*\nPopularity: *{ top_artist_info['popularity']}*\nFollowers: *{top_artist_info['followers']}*")

![Top artist photo](top_artist_photo.jpg)

In [ ]:
min_energy = df['energy'][:5].min()
max_energy = df['energy'][:5].max()

max_loudness = df['loudness'][:5].min()
min_loudness = df['loudness'][:5].max()

In [ ]:
me = sp.me()
my_photo = me['images'][0]['url']
my_country = me['country']
my_name = me['display_name']

In [ ]:
try:
    rec = sp.recommendations(seed_tracks=track_ids[:5], limit=25, min_energy=min_energy, max_energy=max_energy)
except Exception as e:
    # Catch any other exception
    print("An error occurred:", e)

In [ ]:
recommended_ids = get_track_ids(rec, 'tracks')

In [ ]:
rec_tracks = []
for i in range(len(recommended_ids)):
    time.sleep(.5)
    rec_tracks.append(get_track_data(recommended_ids[i])+get_track_features(recommended_ids[i]))

In [ ]:
rec_df = pd.DataFrame(rec_tracks, columns=['name', 'album',
                                   'artist', 'artist_id', 'spotify_url',
                                   'album_cover', 'danceability',
                                   'energy', 'key',
                                   'loudness', 'tempo',
                                   'time_signature'])
rec_df.head(25)

In [ ]:
_, ax = plt.subplots(1, 1)
ax.hist(rec_df['energy'][:5], bins = 5, color = '#FF9300')
ax.set_title("Recommended Energy Distribution")

ax.set_xlabel('energy')
ax.set_ylabel('frequency')

plt.show()

In [ ]:
_, ax = plt.subplots(1, 1)
ax.hist(rec_df['tempo'], bins = 5)
ax.set_title("Tempo Distribution")

ax.set_xlabel('tempo')
ax.set_ylabel('frequency')

plt.show()

In [ ]:
_, ax = plt.subplots(1, 1)
ax.hist(rec_df['key'], bins = [i for i in range(-1, 12)], color = '#FFEC50')
ax.set_title("Key Distribution")

ax.set_xlabel('key')
ax.set_ylabel('frequency')

plt.show()

In [ ]:
_, ax = plt.subplots(1, 1)

ax.hist(df['energy'][:5], bins = 5, alpha=0.5, label='top 5 songs')
ax.hist(rec_df['energy'][:5], bins = 5, alpha=0.5, label='top 5 recommended')
ax.legend(loc='upper right')
ax.set_xlabel('energy')
ax.set_ylabel('num')

plt.show()